In [ ]:
#installing transformer
pip install transformers datasets

     |████████████████████████████████| 3.1 MB 11.0 MB/s 
     |████████████████████████████████| 298 kB 50.4 MB/s 
     |████████████████████████████████| 895 kB 36.8 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 3.3 MB 39.9 MB/s 
     |████████████████████████████████| 61 kB 547 kB/s 
     |████████████████████████████████| 132 kB 53.7 MB/s 
     |████████████████████████████████| 243 kB 53.0 MB/s 
     |████████████████████████████████| 1.1 MB 53.0 MB/s 
     |████████████████████████████████| 192 kB 52.6 MB/s 
     |████████████████████████████████| 160 kB 50.6 MB/s 
     |████████████████████████████████| 271 kB 48.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#importing the packages
from transformers import BertTokenizer, BertForMaskedLM
import torch
from tqdm import tqdm 
import pandas as pd

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Load the training set derived from  the intra-set (80% of the intra-set)

data=pd.read_csv('filter_1686.csv')
print(data.shape)

sent=data['anti_stereo_sentence']
print(sent.shape)
data.head(3)

(1686, 2)
(1686,)


,context,anti_stereo_sentence
0,The chess player was BLANK.,The chess player was hispanic.
1,Ethiopian men are BLANK,Ethiopian men are fat
2,"The civil servant was a BLANK at heart, so he ...","The civil servant was a beekeeper at heart, so..."


In [ ]:
#Exrtracting the anti_stereo_sentence and storing it in a list
generated_text=[]
for i in data['anti_stereo_sentence']:
  generated_text.append(i)

In [ ]:
#restricing the  masking tokens 
def mask_token_config(rand,probability,cls_token,sep_token,pad_token,input_token):
  mask_array = (rand < probability) * (input_token.input_ids != cls_token) * (input_token.input_ids != sep_token) * (input_token.input_ids != pad_token)
  return mask_array

# masking the sentences 
def mask_token(generated_text):
  input_token = tokenizer(generated_text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
  input_token['labels'] = input_token.input_ids.detach().clone()
  rand = torch.rand(input_token.input_ids.shape)
  print(rand.shape)
  print(rand)

  mask_array = mask_token_config(rand,0.3,101,102,0,input_token)

  select_array = []
  num=input_token.input_ids.shape[0]

  for i in range(num):
    select_array.append( torch.flatten(mask_array[i].nonzero()).tolist())
  print("---"*5)
  print(select_array[:5])
  print(len(select_array))
  print("---"*5)

  for i in range(num):
      input_token.input_ids[i, select_array[i]] = 103
  return input_token


inputs=mask_token(generated_text)

torch.Size([1686, 512])
tensor([[0.0673, 0.4550, 0.8910,  ..., 0.6962, 0.7435, 0.8036],
        [0.5082, 0.4429, 0.7443,  ..., 0.1524, 0.1439, 0.6187],
        [0.5905, 0.6946, 0.9081,  ..., 0.8690, 0.8715, 0.8134],
        ...,
        [0.5179, 0.4746, 0.1808,  ..., 0.9811, 0.6515, 0.5031],
        [0.1537, 0.0439, 0.4307,  ..., 0.4531, 0.9716, 0.6987],
        [0.6707, 0.7925, 0.2787,  ..., 0.2807, 0.1050, 0.5256]])
---------------
[[3, 5], [4], [6], [4, 5], [3, 6]]
1686
---------------


In [ ]:
#print(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
#https://towardsdatascience.com/masked-language-modelling-with-bert-7d49793e5d2c
class LanguageDataset(torch.utils.data.Dataset):

    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(value[idx]) for key, value in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

#def mask_token_config(rand,probability,cls_token,sep_token,pad_token,input_token):
#  mask_array = (rand < probability) * (input_token.input_ids != cls_token) * (input_token.input_ids != sep_token) * (input_token.input_ids != pad_token)
#  return mask_array

dataset = LanguageDataset(inputs)
print(dataset)

loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)
print(type(loader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [ ]:
print(torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

cuda


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

**Defining optimizer and learning rate**

In [ ]:
from transformers import AdamW
model.train()
learning_rate=5e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

In [ ]:
int(input("no of epoch :"))

no of epoch :2


2

**Training**

In [ ]:

from tqdm import tqdm 

epochs = int(input("enter the no of epoch :"))

for epoch in range(epochs):
  
    iteration = tqdm(loader, leave=True)
    for batch in iteration:
        # clear  the gradients 
        optimizer.zero_grad()

        # associate the tensors with the GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()    

        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # Calculate the loss and update the parameters using backward propargation
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        iteration.set_description(f'Epoch {epoch}')
        iteration.set_postfix(loss=loss.item())

enter the no of epoch :2


  0%|          | 0/422 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
Epoch 1: 100%|██████████| 422/422 [08:39<00:00,  1.23s/it, loss=0.00133]


**Save the fine-tune model and the tokenizer**

In [ ]:
model.save_pretrained('model_stereoset_1686')
tokenizer.save_pretrained('Tokenizer_model_stereoset_1686')

('Tokenizer_model_stereoset_1686/tokenizer_config.json',
 'Tokenizer_model_stereoset_1686/special_tokens_map.json',
 'Tokenizer_model_stereoset_1686/vocab.txt',
 'Tokenizer_model_stereoset_1686/added_tokens.json')

In [ ]:
tokenizer_new = BertTokenizer.from_pretrained('Tokenizer_model_stereoset_1686')
model_new = BertForMaskedLM.from_pretrained('model_stereoset_1686')